<a href="https://colab.research.google.com/github/wqiu96/summer_project/blob/master/DeepBSDE_pytorch/solver_pytorch_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/wqiu96/summer_project.git

Cloning into 'summer_project'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 663 (delta 8), reused 0 (delta 0), pack-reused 637
Receiving objects: 100% (663/663), 2.55 MiB | 28.64 MiB/s, done.
Resolving deltas: 100% (335/335), done.


In [2]:
cd summer_project/DeepBSDE_pytorch/

/content/summer_project/DeepBSDE_pytorch


In [0]:
import logging
import time
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.autograd import Variable
from torch.distributions import uniform
import torchvision
from equation_pytorch import get_equation
from config_pytorch import get_config


MOMENTUM = 0.99
EPSILON = 1e-6
DELTA_CLIP = 50.0


class Net(nn.Module):
    def __init__(self,num_hiddens):
      super(Net, self).__init__()
      self.num_hiddens = num_hiddens
      
      self.fc1 = nn.Linear(num_hiddens[0], num_hiddens[1], bias=False)
      self.norm1 = nn.LayerNorm(num_hiddens[1])
      #self.fc2 = nn.Linear(num_hiddens[1], num_hiddens[2], bias=False)
      #self.norm2 = nn.LayerNorm(num_hiddens[2])
      #self.fc3 = nn.Linear(num_hiddens[2], num_hiddens[3], bias=False)
      self.fc3 = nn.Linear(num_hiddens[1], num_hiddens[2], bias=False)
    def forward(self, x):
      # h1 = relu(xw1)
      x = self.norm1(F.relu(self.fc1(x)))
      #x = self.norm1(self.fc1(x))
      # h2 = relu(h1w2)
      #x = self.norm2(self.fc2(x))
      #x = self.norm1(F.relu(self.fc2(x)))
      # h3 = h2w3
      x = self.fc3(x)
      #termin time
      return x
      

class DeepNet(nn.Module):
    def __init__(self,num_hiddens,config,bsde):
      super(DeepNet, self).__init__()
      self.num_hiddens = num_hiddens
      self._config = config
      self._bsde = bsde
      self.y_init = 0
      self._dim = bsde.dim
      self._num_time_interval = bsde.num_time_interval
      self.linears = nn.ModuleList([Net(num_hiddens) for i in range(bsde.num_time_interval - 1)])
      m = uniform.Uniform(config.y_init_range[0],config.y_init_range[1])
      self.y_init = torch.nn.Parameter(m.sample())
      self.z_init = torch.nn.Parameter(2*torch.rand([self._dim,1],dtype=torch.float32) - 1)

    
    def forward(self,x):
      dw_train= self._bsde.sample()[0].astype(np.float32)
      time_stamp = np.arange(0, self._bsde.num_time_interval) * self._bsde.delta_t
      z = self.z_init
      y_ = self.y_init
      for t in range(0,bsde.num_time_interval - 1):
        dw = torch.from_numpy(dw_train[:, t]).view(1,self._dim)
        torch.mm(dw.float(), z.float()) # torch.mm have bug use x.float()
        y_ = y_ - self._bsde.delta_t* (self._bsde.f_tf(time_stamp[t], x[:, t], y_, z)) + torch.mm(dw, z)
        z = (self.linears[t](x[:,t]) / self._dim).view(self._dim,1)
      #terminal condition
      dw = torch.from_numpy(dw_train[:, -1]).view(1,self._dim)
      y_ = y_ - self._bsde.delta_t * (self._bsde.f_tf(time_stamp[-1], x[:, -2], y_, z)) + torch.mm(dw, z)
      return y_


Result:
- AllenCahn: sometimes my code can give the similar result as the original code, sometimes the result is nan(because the loss is too small to learn??).
- HJB: sometimes have the same result as paper,sometimes not
- PricingOption: about 21
- PricingDefaultRisk: about 56
- BurgesType: about 1.2
- QuadraticGradients: about 2.55 but the paper's result is 0.8
- ReactionDiffusion : Multiple results compared with the paper

In [7]:
a = []
for j in range(10):
  config = get_config('HJB')
  bsde = get_equation('HJB', config.dim, config.total_time, config.num_time_interval)
  deepNet = DeepNet(config.num_hiddens,config,bsde)
  optimizer = optim.SGD(deepNet.parameters(), lr=0.00001, momentum=MOMENTUM) # lr have some different wiht the original
  #torch.optim.lr_scheduler.MultiStepLR(optimizer, [15,25,35], gamma=0.1, last_epoch=-1) # Adjust learning rate according to time
  train_loss = []
  for epoch in range(10000):
    x_ = bsde.sample()[1].astype(np.float32)
    x = torch.from_numpy(x_)
    out = deepNet(x)
    delta = out - bsde.g_tf(bsde.total_time, x[:, -1])
    #loss = torch.mean(torch.where(torch.abs(delta) < DELTA_CLIP, torch.pow(delta,2),2 * DELTA_CLIP * torch.abs(delta) - DELTA_CLIP ** 2))
    loss = delta**2
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
    #if epoch % 100 == 0 :
      #print(epoch, loss.item(), deepNet.y_init)
    if loss <= 0.00001:
      a.append(deepNet.y_init)
      break
sum(a)/10

tensor(0.5973, grad_fn=<DivBackward0>)